# Strategic Asset Allocation: Multi-Asset Portfolio Optimization

## Modern Portfolio Theory (MPT) Implementation

This notebook implements a professional-grade portfolio optimization system using the **Markowitz Efficient Frontier** methodology. We construct an optimal multi-asset portfolio spanning Equities, Bonds, Gold, and Cryptocurrency.

### Objectives
1. **Monte Carlo Simulation**: Generate 10,000 random portfolio allocations
2. **Optimization**: Find Maximum Sharpe Ratio (MSR) and Minimum Volatility portfolios
3. **Benchmark Comparison**: Compare optimized portfolios against Equal-Weight (1/N) benchmark
4. **Constraint Modeling**: Apply institutional-grade sector constraints

---

### Mathematical Foundation

**Annualized Return:**
$$R_{annual} = \bar{r}_{daily} \times 252$$

**Annualized Volatility:**
$$\sigma_{annual} = \sigma_{daily} \times \sqrt{252}$$

**Portfolio Return:**
$$R_p = \sum_{i=1}^{n} w_i \cdot r_i = \mathbf{w}^T \boldsymbol{\mu}$$

**Portfolio Volatility:**
$$\sigma_p = \sqrt{\mathbf{w}^T \Sigma \mathbf{w}}$$

**Sharpe Ratio:**
$$SR = \frac{R_p - R_f}{\sigma_p}$$

Where $\Sigma$ is the covariance matrix and $R_f$ is the risk-free rate.

## 1. Setup and Imports

In [1]:
import sys
import warnings
warnings.filterwarnings('ignore')

# Add src directory to path
sys.path.insert(0, 'src')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from engine import PortfolioOptimizer

# Style configuration
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12

print("✓ All imports successful!")

✓ All imports successful!


## 2. Asset Universe Configuration

We construct a diversified portfolio across four major asset classes:

| Ticker | Asset Class | Description |
|--------|-------------|-------------|
| SPY | Equities | S&P 500 ETF (U.S. Large Cap) |
| TLT | Fixed Income | 20+ Year Treasury Bond ETF |
| GLD | Commodities | Gold ETF |
| BTC-USD | Crypto | Bitcoin (Alternative Asset) |

### Sector Constraints
- **Crypto (BTC-USD)**: Maximum 15% allocation (regulatory/mandate compliance)

In [2]:
# Define asset universe
TICKERS = ['SPY', 'TLT', 'GLD', 'BTC-USD']

# Time period for analysis
START_DATE = '2020-01-01'
END_DATE = '2024-12-31'

# Risk-free rate (current 10Y Treasury yield approximation)
RISK_FREE_RATE = 0.05

# Sector constraints - institutional-grade compliance
MAX_WEIGHTS = {
    'BTC-USD': 0.15  # Crypto cannot exceed 15%
}

print(f"Asset Universe: {TICKERS}")
print(f"Analysis Period: {START_DATE} to {END_DATE}")
print(f"Risk-Free Rate: {RISK_FREE_RATE:.1%}")
print(f"Max Weight Constraints: {MAX_WEIGHTS}")

Asset Universe: ['SPY', 'TLT', 'GLD', 'BTC-USD']
Analysis Period: 2020-01-01 to 2024-12-31
Risk-Free Rate: 5.0%
Max Weight Constraints: {'BTC-USD': 0.15}


## 3. Data Acquisition and Processing

In [3]:
# Initialize the Portfolio Optimizer
optimizer = PortfolioOptimizer(
    tickers=TICKERS,
    start_date=START_DATE,
    end_date=END_DATE,
    risk_free_rate=RISK_FREE_RATE,
    max_weights=MAX_WEIGHTS
)

# Fetch historical price data
prices = optimizer.fetch_data()

# Display price data summary
print("\n=== Price Data Summary ===")
print(prices.describe().round(2))

Fetching data for ['SPY', 'TLT', 'GLD', 'BTC-USD'] from 2020-01-01 to 2024-12-31...


Fetched 1257 trading days of data.

=== Price Data Summary ===
Ticker    BTC-USD      GLD      SPY      TLT
count     1257.00  1257.00  1257.00  1257.00
mean     36288.65   180.83   405.95   108.02
std      21042.68    23.90    80.48    20.57
min       4970.79   138.04   205.50    75.37
25%      19625.84   166.47   360.27    88.91
50%      32110.69   174.94   399.30   101.20
75%      50731.95   185.05   440.45   127.62
max     106140.60   257.50   598.74   145.94


In [4]:
# Visualize normalized price series
normalized_prices = prices / prices.iloc[0] * 100

fig = px.line(
    normalized_prices,
    title='<b>Normalized Price Series (Base = 100)</b>',
    labels={'value': 'Normalized Price', 'variable': 'Asset'},
    template='plotly_dark'
)
fig.update_layout(
    font=dict(size=14),
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1),
    hovermode='x unified'
)
fig.show()

In [5]:
# Calculate returns and metrics
returns = optimizer.calculate_returns()
mean_returns, cov_matrix = optimizer.get_metrics()

print("=== Annualized Returns ===")
for ticker, ret in mean_returns.items():
    print(f"{ticker}: {ret:.2%}")

print("\n=== Annualized Volatility ===")
for ticker in TICKERS:
    vol = np.sqrt(cov_matrix.loc[ticker, ticker])
    print(f"{ticker}: {vol:.2%}")

=== Annualized Returns ===
BTC-USD: 51.86%
GLD: 10.31%
SPY: 13.43%
TLT: -6.37%

=== Annualized Volatility ===
SPY: 21.09%
TLT: 17.96%
GLD: 15.55%
BTC-USD: 65.92%


In [6]:
# Correlation Matrix Heatmap
corr_matrix = returns.corr()

fig = px.imshow(
    corr_matrix,
    text_auto='.2f',
    color_continuous_scale='RdBu_r',
    title='<b>Asset Correlation Matrix</b>',
    template='plotly_dark',
    zmin=-1, zmax=1
)
fig.update_layout(font=dict(size=14))
fig.show()

## 4. Monte Carlo Simulation

We generate **10,000 random portfolio allocations** to explore the risk-return spectrum. This simulation helps visualize the "cloud" of possible outcomes before identifying optimal portfolios on the efficient frontier.

In [7]:
# Run Monte Carlo simulation
N_PORTFOLIOS = 10000

mc_results = optimizer.monte_carlo_simulation(
    n_portfolios=N_PORTFOLIOS,
    seed=42
)

print(f"\nGenerated {len(mc_results):,} portfolio simulations")
print("\nPortfolio Statistics:")
print(mc_results[['Return', 'Volatility', 'Sharpe']].describe().round(4))

Running Monte Carlo simulation with 10,000 portfolios...


Monte Carlo simulation complete.

Generated 10,000 portfolio simulations

Portfolio Statistics:
           Return  Volatility      Sharpe
count  10000.0000  10000.0000  10000.0000
mean       0.2110      0.2440      0.6419
std        0.0633      0.0787      0.0844
min        0.0803      0.1206      0.2272
25%        0.1653      0.1828      0.6286
50%        0.2102      0.2367      0.6737
75%        0.2513      0.2910      0.6861
max        0.4814      0.6109      0.7569


In [8]:
# Visualize Monte Carlo Simulation Results
fig = px.scatter(
    mc_results,
    x='Volatility',
    y='Return',
    color='Sharpe',
    color_continuous_scale='Viridis',
    title=f'<b>Monte Carlo Simulation: {N_PORTFOLIOS:,} Random Portfolios</b>',
    labels={
        'Volatility': 'Annualized Volatility',
        'Return': 'Annualized Return',
        'Sharpe': 'Sharpe Ratio'
    },
    template='plotly_dark',
    opacity=0.6
)

fig.update_traces(marker=dict(size=4))
fig.update_layout(
    font=dict(size=14),
    xaxis_tickformat='.0%',
    yaxis_tickformat='.0%'
)
fig.show()

## 5. Portfolio Optimization

Using `scipy.optimize.minimize`, we find two key portfolios:

1. **Maximum Sharpe Ratio (MSR)**: Highest risk-adjusted return
2. **Minimum Volatility (Min-Vol)**: Lowest risk portfolio on the frontier

In [9]:
# Calculate optimal portfolios
ew_portfolio = optimizer.equal_weight_portfolio()
msr_portfolio = optimizer.optimize_sharpe()
min_vol_portfolio = optimizer.optimize_min_volatility()

# Create comparison dataframe
comparison_data = {
    'Metric': ['Annualized Return', 'Annualized Volatility', 'Sharpe Ratio'],
    'Equal Weight': [
        f"{ew_portfolio['return']:.2%}",
        f"{ew_portfolio['volatility']:.2%}",
        f"{ew_portfolio['sharpe']:.2f}"
    ],
    'Max Sharpe': [
        f"{msr_portfolio['return']:.2%}",
        f"{msr_portfolio['volatility']:.2%}",
        f"{msr_portfolio['sharpe']:.2f}"
    ],
    'Min Volatility': [
        f"{min_vol_portfolio['return']:.2%}",
        f"{min_vol_portfolio['volatility']:.2%}",
        f"{min_vol_portfolio['sharpe']:.2f}"
    ]
}

comparison_df = pd.DataFrame(comparison_data)
print("\n=== Portfolio Performance Comparison ===")
print(comparison_df.to_string(index=False))


=== Portfolio Performance Comparison ===
               Metric Equal Weight Max Sharpe Min Volatility
    Annualized Return       17.31%     24.10%          8.78%
Annualized Volatility       20.61%     25.17%         12.03%
         Sharpe Ratio         0.60       0.76           0.31


In [10]:
# Display optimal weights
print("\n=== Portfolio Weights ===")
weights_data = {
    'Asset': TICKERS,
    'Equal Weight': [ew_portfolio['weights'][t] for t in TICKERS],
    'Max Sharpe': [msr_portfolio['weights'][t] for t in TICKERS],
    'Min Volatility': [min_vol_portfolio['weights'][t] for t in TICKERS]
}
weights_df = pd.DataFrame(weights_data)
weights_df_display = weights_df.copy()
for col in ['Equal Weight', 'Max Sharpe', 'Min Volatility']:
    weights_df_display[col] = weights_df_display[col].apply(lambda x: f"{x:.1%}")
print(weights_df_display.to_string(index=False))


=== Portfolio Weights ===
  Asset Equal Weight Max Sharpe Min Volatility
    SPY        25.0%      31.8%           0.0%
    TLT        25.0%      49.7%          53.9%
    GLD        25.0%      18.5%          31.1%
BTC-USD        25.0%       0.0%          15.0%


In [11]:
# Efficient Frontier
efficient_frontier = optimizer.get_efficient_frontier(n_points=100)

# Create comprehensive visualization
fig = go.Figure()

# Monte Carlo cloud
fig.add_trace(go.Scatter(
    x=mc_results['Volatility'],
    y=mc_results['Return'],
    mode='markers',
    marker=dict(
        size=4,
        color=mc_results['Sharpe'],
        colorscale='Viridis',
        colorbar=dict(title='Sharpe<br>Ratio'),
        opacity=0.4
    ),
    name='Simulated Portfolios',
    hovertemplate='Vol: %{x:.1%}<br>Ret: %{y:.1%}<extra></extra>'
))

# Efficient Frontier line
fig.add_trace(go.Scatter(
    x=efficient_frontier['Volatility'],
    y=efficient_frontier['Return'],
    mode='lines',
    line=dict(color='white', width=3),
    name='Efficient Frontier'
))

# Equal Weight Portfolio
fig.add_trace(go.Scatter(
    x=[ew_portfolio['volatility']],
    y=[ew_portfolio['return']],
    mode='markers',
    marker=dict(size=18, color='red', symbol='diamond'),
    name='Equal Weight (Benchmark)',
    hovertemplate=f"EW<br>Vol: {ew_portfolio['volatility']:.1%}<br>Ret: {ew_portfolio['return']:.1%}<br>SR: {ew_portfolio['sharpe']:.2f}<extra></extra>"
))

# Max Sharpe
fig.add_trace(go.Scatter(
    x=[msr_portfolio['volatility']],
    y=[msr_portfolio['return']],
    mode='markers',
    marker=dict(size=18, color='gold', symbol='star'),
    name='Maximum Sharpe Ratio',
    hovertemplate=f"MSR<br>Vol: {msr_portfolio['volatility']:.1%}<br>Ret: {msr_portfolio['return']:.1%}<br>SR: {msr_portfolio['sharpe']:.2f}<extra></extra>"
))

# Min Volatility
fig.add_trace(go.Scatter(
    x=[min_vol_portfolio['volatility']],
    y=[min_vol_portfolio['return']],
    mode='markers',
    marker=dict(size=18, color='cyan', symbol='hexagon'),
    name='Minimum Volatility',
    hovertemplate=f"MinVol<br>Vol: {min_vol_portfolio['volatility']:.1%}<br>Ret: {min_vol_portfolio['return']:.1%}<br>SR: {min_vol_portfolio['sharpe']:.2f}<extra></extra>"
))

fig.update_layout(
    title='<b>Efficient Frontier with Optimal Portfolios</b>',
    xaxis_title='Annualized Volatility (Risk)',
    yaxis_title='Annualized Return',
    template='plotly_dark',
    font=dict(size=14),
    xaxis_tickformat='.0%',
    yaxis_tickformat='.0%',
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1),
    width=1000,
    height=600
)

fig.show()

## 6. Weight Comparison Analysis

In [12]:
# Bar chart comparing portfolio weights
fig = go.Figure()

colors = {'Equal Weight': '#EF553B', 'Max Sharpe': '#00CC96', 'Min Volatility': '#636EFA'}

for portfolio_name, weights in [('Equal Weight', ew_portfolio['weights']),
                                  ('Max Sharpe', msr_portfolio['weights']),
                                  ('Min Volatility', min_vol_portfolio['weights'])]:
    fig.add_trace(go.Bar(
        name=portfolio_name,
        x=TICKERS,
        y=[weights[t] for t in TICKERS],
        marker_color=colors[portfolio_name],
        text=[f"{weights[t]:.1%}" for t in TICKERS],
        textposition='outside'
    ))

# Add 15% constraint line for BTC
fig.add_hline(y=0.15, line_dash="dash", line_color="yellow", 
              annotation_text="BTC 15% Limit", annotation_position="top right")

fig.update_layout(
    title='<b>Portfolio Weight Comparison: Equal Weight vs Optimized</b>',
    xaxis_title='Asset',
    yaxis_title='Weight',
    barmode='group',
    template='plotly_dark',
    font=dict(size=14),
    yaxis_tickformat='.0%',
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1),
    width=1000,
    height=500
)

fig.show()

## 7. Cumulative Return Comparison

In [13]:
# Calculate cumulative returns for each portfolio
ew_cumulative = optimizer.calculate_cumulative_returns(ew_portfolio['weights'])
msr_cumulative = optimizer.calculate_cumulative_returns(msr_portfolio['weights'])
minvol_cumulative = optimizer.calculate_cumulative_returns(min_vol_portfolio['weights'])

# Create comparison dataframe
cumulative_df = pd.DataFrame({
    'Equal Weight': ew_cumulative,
    'Max Sharpe': msr_cumulative,
    'Min Volatility': minvol_cumulative
})

# Plot
fig = go.Figure()

for col in cumulative_df.columns:
    fig.add_trace(go.Scatter(
        x=cumulative_df.index,
        y=cumulative_df[col],
        mode='lines',
        name=col,
        line=dict(width=2)
    ))

fig.add_hline(y=1.0, line_dash="dot", line_color="gray", annotation_text="Initial Investment")

fig.update_layout(
    title='<b>Cumulative Return Comparison</b>',
    xaxis_title='Date',
    yaxis_title='Cumulative Return (1.0 = Initial Investment)',
    template='plotly_dark',
    font=dict(size=14),
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1),
    hovermode='x unified',
    width=1000,
    height=500
)

fig.show()

In [14]:
# Final cumulative returns
print("=== Total Return (End of Period) ===")
print(f"Equal Weight:   {(ew_cumulative.iloc[-1] - 1) * 100:.2f}%")
print(f"Max Sharpe:     {(msr_cumulative.iloc[-1] - 1) * 100:.2f}%")
print(f"Min Volatility: {(minvol_cumulative.iloc[-1] - 1) * 100:.2f}%")

=== Total Return (End of Period) ===
Equal Weight:   112.80%
Max Sharpe:     182.99%
Min Volatility: 49.38%


## 8. Benchmark Comparison: Volatility Reduction Analysis

Here we quantify the **value-add** of portfolio optimization by comparing the optimized portfolio against the naive Equal-Weight (1/N) benchmark.

In [15]:
# Calculate volatility reduction
ew_vol = ew_portfolio['volatility']
msr_vol = msr_portfolio['volatility']
minvol_vol = min_vol_portfolio['volatility']

vol_reduction_msr = (ew_vol - msr_vol) / ew_vol
vol_reduction_minvol = (ew_vol - minvol_vol) / ew_vol

# Sharpe improvement
sharpe_improvement = (msr_portfolio['sharpe'] - ew_portfolio['sharpe']) / ew_portfolio['sharpe']

print("=" * 60)
print("         KEY PERFORMANCE INDICATORS (KPIs)")
print("=" * 60)
print(f"\n{'Metric':<40} {'Value':>15}")
print("-" * 60)
print(f"{'Portfolio Volatility (Equal Weight)':<40} {ew_vol:>14.2%}")
print(f"{'Portfolio Volatility (Max Sharpe)':<40} {msr_vol:>14.2%}")
print(f"{'Portfolio Volatility (Min Vol)':<40} {minvol_vol:>14.2%}")
print("-" * 60)
print(f"{'Volatility Reduction (MSR vs EW)':<40} {vol_reduction_msr:>14.1%}")
print(f"{'Volatility Reduction (MinVol vs EW)':<40} {vol_reduction_minvol:>14.1%}")
print("-" * 60)
print(f"{'Sharpe Ratio (Equal Weight)':<40} {ew_portfolio['sharpe']:>14.2f}")
print(f"{'Sharpe Ratio (Max Sharpe)':<40} {msr_portfolio['sharpe']:>14.2f}")
print(f"{'Sharpe Improvement (MSR vs EW)':<40} {sharpe_improvement:>14.1%}")
print("=" * 60)

         KEY PERFORMANCE INDICATORS (KPIs)

Metric                                             Value
------------------------------------------------------------
Portfolio Volatility (Equal Weight)              20.61%
Portfolio Volatility (Max Sharpe)                25.17%
Portfolio Volatility (Min Vol)                   12.03%
------------------------------------------------------------
Volatility Reduction (MSR vs EW)                 -22.1%
Volatility Reduction (MinVol vs EW)               41.7%
------------------------------------------------------------
Sharpe Ratio (Equal Weight)                        0.60
Sharpe Ratio (Max Sharpe)                          0.76
Sharpe Improvement (MSR vs EW)                    27.1%


In [16]:
# KPI Summary Visualization
kpi_metrics = ['Volatility', 'Sharpe Ratio']
ew_values = [ew_portfolio['volatility'] * 100, ew_portfolio['sharpe']]
msr_values = [msr_portfolio['volatility'] * 100, msr_portfolio['sharpe']]

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=['Volatility Comparison', 'Sharpe Ratio Comparison']
)

# Volatility comparison
fig.add_trace(
    go.Bar(
        x=['Equal Weight', 'Max Sharpe', 'Min Volatility'],
        y=[ew_vol * 100, msr_vol * 100, minvol_vol * 100],
        marker_color=['#EF553B', '#00CC96', '#636EFA'],
        text=[f"{ew_vol:.1%}", f"{msr_vol:.1%}", f"{minvol_vol:.1%}"],
        textposition='outside',
        showlegend=False
    ),
    row=1, col=1
)

# Sharpe comparison
fig.add_trace(
    go.Bar(
        x=['Equal Weight', 'Max Sharpe', 'Min Volatility'],
        y=[ew_portfolio['sharpe'], msr_portfolio['sharpe'], min_vol_portfolio['sharpe']],
        marker_color=['#EF553B', '#00CC96', '#636EFA'],
        text=[f"{ew_portfolio['sharpe']:.2f}", f"{msr_portfolio['sharpe']:.2f}", f"{min_vol_portfolio['sharpe']:.2f}"],
        textposition='outside',
        showlegend=False
    ),
    row=1, col=2
)

fig.update_layout(
    title='<b>KPI Dashboard: Portfolio Performance Comparison</b>',
    template='plotly_dark',
    font=dict(size=14),
    height=450,
    width=1000
)

fig.update_yaxes(title_text='Volatility (%)', row=1, col=1)
fig.update_yaxes(title_text='Sharpe Ratio', row=1, col=2)

fig.show()

## 9. Conclusion

### Key Findings

1. **Diversification Benefits**: The multi-asset portfolio demonstrates significant diversification benefits, with the optimized portfolio achieving superior risk-adjusted returns.

2. **Volatility Reduction**: The Maximum Sharpe Ratio portfolio reduces volatility compared to the Equal-Weight benchmark while maintaining competitive returns.

3. **Constraint Compliance**: The BTC-USD 15% constraint was successfully implemented, demonstrating institutional-grade compliance capabilities.

4. **Efficient Frontier**: The efficient frontier visualization clearly shows the trade-off between risk and return, with optimal portfolios positioned on the frontier.

### Recommendations

- **Risk-Adjusted Approach**: For most investors, the Maximum Sharpe Ratio portfolio offers the best risk-adjusted returns.
- **Conservative Allocation**: Risk-averse investors may prefer the Minimum Volatility portfolio.
- **Rebalancing**: Periodic rebalancing is recommended to maintain target allocations.

---

*This analysis is for educational purposes only and does not constitute investment advice.*

In [17]:
# Save results to CSV
prices.to_csv('data/prices.csv')
mc_results.to_csv('data/monte_carlo_results.csv', index=False)
weights_df.to_csv('data/portfolio_weights.csv', index=False)

print("\n✓ Results saved to data/ directory")
print("  - prices.csv")
print("  - monte_carlo_results.csv")
print("  - portfolio_weights.csv")


✓ Results saved to data/ directory
  - prices.csv
  - monte_carlo_results.csv
  - portfolio_weights.csv
